In [1]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import matplotlib as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [ ]:
business = pd.read_json('yelp_dataset/business.json', lines=True)
#checkin = pd.read_json('yelp_dataset/checkin.json', lines=True)
#photo = pd.read_json('yelp_dataset/photo.json', lines=True)
review = pd.read_json('yelp_dataset/review.json', lines=True)
#tip = pd.read_json('yelp_dataset/tip.json', lines=True)
##user = pd.read_json('yelp_dataset/user.json', lines=True)

In [ ]:
review.head()

### Only look at restaurants

In [3]:
#lv_restaurants
restaurants = business['categories'].str.contains('Restaurants', regex=False)
restaurants = restaurants.fillna(False)
restaurants = business[restaurants]
print(restaurants.shape)
restaurants.head()

(59371, 14)


,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
11,2450 E Indian School Rd,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1Dfx3zM-rW4n-31KeC8sJg,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",1,33.495194,-112.028588,Taco Bell,85016,18,3.0,AZ
13,5981 Andrews Rd,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...",fweCYi8FmbJXHCqLnwuk8w,"Italian, Restaurants, Pizza, Chicken Wings",Mentor-on-the-Lake,"{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",1,41.708520,-81.359556,Marco's Pizza,44060,16,4.0,OH
17,"1775 E Tropicana Ave, Ste 29","{'OutdoorSeating': 'False', 'BusinessAcceptsCr...",PZ-LZzSlhSe9utkQYU8pFg,"Restaurants, Italian",Las Vegas,None,0,36.100016,-115.128529,Carluccio's Tivoli Gardens,89119,40,4.0,NV


In [4]:
X_train, X_test, y_train, y_test = train_test_split(restaurants['categories'], \
                                                    restaurants['stars'], test_size=0.33, random_state=42)


In [5]:
X_train = pd.Series(X_train).str.get_dummies(sep=', ')
X_test = pd.Series(X_test).str.get_dummies(sep=', ')

In [7]:
### Vectorized value to get categories
#vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1))
#categories_count = vectorizer.fit_transform(restaurants['categories']).astype(np.int8)
#categories_list = vectorizer.get_feature_names()
#categories_df = pd.DataFrame(categories_count.toarray())
#categories_df.columns = categories_list
#categories_df.head()

,acai,accessories,accountants,acne,active,activities,acupuncture,adoption,adult,advertising,...,wineries,wings,women,wraps,yelp,yoga,yogurt,your,yourself,zoos
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_test.head()

,& Probates,Acai Bowls,Accessories,Acne Treatment,Active Life,Adult Entertainment,Afghan,African,Air Duct Cleaning,Aircraft Repairs,...,Wine & Spirits,Wine Bars,Wine Tasting Classes,Wine Tours,Wineries,Women's Clothing,Wraps,Yelp Events,Yoga,Zoos
67682,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
56469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88792,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#check for columns missing from test set
missing_train = list(set(list(X_train)) - set(list(X_test)))
for col in missing_train:
    X_test[col] = 0
    
missing_test = list(set(list(X_test)) - set(list(X_train)))
for col in missing_test:
    X_train[col] = 0

In [10]:
X_train.head()

,Acai Bowls,Accessories,Accountants,Active Life,Acupuncture,Adult Education,Adult Entertainment,Advertising,Afghan,African,...,Metal Fabricators,Rest Stops,Employment Agencies,Mortgage Brokers,Preschools,RV Parks,Game Meat,Gardeners,Carpet Installation,Auto Insurance
68223,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79510,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42070,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48622,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X_test.head()

,& Probates,Acai Bowls,Accessories,Acne Treatment,Active Life,Adult Entertainment,Afghan,African,Air Duct Cleaning,Aircraft Repairs,...,Art Supplies,Scottish,Historical Tours,Printing Services,Mass Media,Service Stations,Senegalese,Session Photography,Marinas,Pet Training
67682,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88792,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
print(X_train.shape)
print(X_test.shape)

(39778, 761)
(19593, 761)


In [16]:
y_train_cut = pd.cut(np.array(y_train), 2, labels=[1, 2])
y_test_cut = pd.cut(np.array(y_test), 2, labels=[1, 2])
np.array(y_train_cut)

array([2, 1, 2, ..., 1, 2, 2])

In [17]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, np.array(y_train_cut)) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [18]:
np.array(y_test_cut)

array([2, 2, 2, ..., 1, 1, 2])

In [19]:
neigh.score(X_test, np.array(y_test_cut))

0.5274332669831062

In [73]:
y_pred = neigh.predict(X_test)

In [74]:
accuracy_score(y_test_cut, y_pred)

0.5274332669831062

## Linear regression

#### Ridge

In [56]:
reg = linear_model.Ridge(alpha=.5)

In [57]:
reg.fit(X_train, y_train)


Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [58]:
y_pred = reg.predict(X_test)
y_pred = pd.Series(y_pred)

In [59]:
y_pred.shape

(19593,)

In [60]:
y_test.shape

(19593,)

In [61]:
mean_squared_error(y_test, y_pred)

1.568242542272463

In [62]:
r2_score(y_test, y_pred)  

-1.4283457838421336

#### Standard

In [66]:
regL = linear_model.LinearRegression()

In [67]:
regL.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [68]:
y_pred = regL.predict(X_test)
y_pred = pd.Series(y_pred)

In [69]:
y_pred.shape

(19593,)

In [70]:
y_test.shape

(19593,)

In [71]:
mean_squared_error(y_test, y_pred)

1.9275903467365447e+24

In [72]:
r2_score(y_test, y_pred)  

-2.984778033562134e+24